In [1]:
from dotenv import load_dotenv
import google.generativeai as genai

import os
from src.services.llm_generation.llm import Prompt
from src.services.tools.News.news import API_KEY
from src.services.tools.agent_tools import AgentTools
from openai import OpenAI

load_dotenv()
openai_api_key = os.getenv("openai_key")
gemini_api_key = os.getenv("gemini_key")
claude_api_key = os.getenv("claude_key")
tools_path = os.getenv("tools")

gemini_model="gemini-2.5-flash"
llm_model=gemini_model
gemini_key=gemini_api_key
api_key = gemini_key
gemini_base_url= "https://generativelanguage.googleapis.com/v1beta"






model=OpenAI(api_key=gemini_key,base_url=gemini_base_url)
tools_openai, tools_gemini = AgentTools().list_of_tools(tools_path)
role="user"
context=""
source=""
message ="quelle est la capitale de la Belgique?"
prompt=Prompt(role,context,source,message)

C:\Users\tallar\Documents\PROJETS\GenAI\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

print(tools_gemini)
 # récupère la clé depuis GEMINI_API_KEY ou GOOGLE_API_KEY


[{'function_declarations': [{'name': 'list_of_tools', 'description': 'No description available.', 'parameters': {'type': 'object', 'properties': {'self': {'type': 'string', 'description': 'Paramètre self'}, 'racine': {'type': 'string', 'description': 'Paramètre racine'}}, 'required': ['self', 'racine']}}]}, {'function_declarations': [{'name': 'f_get_data', 'description': ':param id: identifiant du redevable\n:return: la liste des créances ou des transactions encore ourvertes', 'parameters': {'type': 'object', 'properties': {'id': {'type': 'string', 'description': 'Paramètre id'}}, 'required': ['id']}}]}, {'function_declarations': [{'name': 'f_get_weather', 'description': "Récupère la météo actuelle en fonction de la latitude et longitude via l'API Open-Meteo.\n\nArgs:\n    latitude (float): Latitude de la position.\n    longitude (float): Longitude de la position.\n\nReturns:\n    str: Description simple de la météo (température, conditions).", 'parameters': {'type': 'object', 'propert

In [25]:

genai.configure(api_key=gemini_api_key)
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash", # Replace with your actual key
    tools=tools_gemini
)
# Generate content
message ="donne une liste des transactions du redevable VDL-000030821?"
response = model.generate_content(message)

results=[f"la question initiale: {message}"]
tool_responses=[]
# Print the result
for part in response.candidates[0].content.parts:
    # Vérifier si la partie est un appel de fonction
    if hasattr(part, 'function_call'):
        tool_call = part.function_call

        # Obtenir le nom de la fonction et ses arguments
        tool_name = tool_call.name
        tool_args = tool_call.args

        print(f"Le modèle a demandé d'exécuter la fonction : {tool_name}")
        print(f"Avec les arguments : {tool_args}")
        result = AgentTools().use_tool(tool_name,tool_args)

        tool_responses.append(result)

if tool_responses:
    print("\n-------------------------------------------------------------")
    print("Résultat des outils obtenu. On le renvoie au modèle...")
    print("-------------------------------------------------------------")

    # 1. Créez un objet de contenu pour le message initial de l'utilisateur
    user_content = genai.protos.Content(
        parts=[genai.protos.Part(text=message)],
        role='user'
    )

    # 2. L'historique de la conversation commence par le message de l'utilisateur...
    chat_history = [user_content]

    # 3. ...suivi par la réponse du modèle qui contient l'appel de fonction...
    chat_history.append(response.candidates[0].content)

    # 4. ...suivi par la réponse de l'outil qui contient les résultats.
    tool_content = genai.protos.Content(
        parts=[genai.protos.Part(text=tool_responses[0])],
        role='user'
    )
    chat_history.append(tool_content)

    # Relancez generate_content avec l'historique complet pour obtenir la réponse finale
    final_response = model.generate_content(chat_history)
    print("\n-------------------------------------------------------------")
    print("Réponse finale du modèle :")
    print("-------------------------------------------------------------")
    print(final_response.text)
else:
    print("\n-------------------------------------------------------------")
    print("Le modèle a fourni une réponse textuelle directe :")
    print("-------------------------------------------------------------")
    print(response.text)


Le modèle a demandé d'exécuter la fonction : f_get_data
Avec les arguments : <proto.marshal.collections.maps.MapComposite object at 0x000001E4FEF46B90>

-------------------------------------------------------------
Résultat des outils obtenu. On le renvoie au modèle...
-------------------------------------------------------------

-------------------------------------------------------------
Réponse finale du modèle :
-------------------------------------------------------------
J'ai bien noté que le redevable VDL-000030821 a 2 créances ouvertes avec les références 152011989007 et 240006105142, toutes deux liées aux redevances de stationnement du 31/05/2022.


AttributeError: 'Client' object has no attribute 'responses'